In [9]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import gamma
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv(r"C:\Users\Zery Chan\Downloads\CL.csv")

In [11]:
df

,datetime,open,high,low,close,volume,range,range_up,range_dn,delta_close,volume_ewma,volume_ewmv,range_ewma,range_ewmv,volume_state,vol_state,trend_state
0,2020-03-05 00:00:00,47.45,47.47,47.42,47.44,93,0.05,0.02,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-05 00:05:00,47.45,47.45,47.43,47.45,12,0.02,0.00,0.02,0.01,93.000000,0.000000,0.050000,0.000000,0.0,0.0,1.0
2,2020-03-05 00:10:00,47.46,47.53,47.46,47.53,12,0.07,0.07,0.00,0.08,51.096939,28.120505,0.034480,0.010415,0.0,0.0,2.0
3,2020-03-05 00:15:00,47.52,47.55,47.52,47.54,63,0.03,0.03,0.00,0.01,37.151594,27.098879,0.047150,0.016001,0.0,0.0,1.0
4,2020-03-05 00:20:00,47.54,47.57,47.52,47.57,22,0.05,0.03,0.02,0.03,44.300261,25.059224,0.042407,0.015093,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11034,2020-05-14 23:35:00,27.75,27.85,27.75,27.77,65,0.10,0.10,0.00,0.01,29.617817,82.909647,0.060278,0.030883,0.0,0.0,1.0
11035,2020-05-14 23:40:00,27.77,27.80,27.75,27.75,8,0.05,0.03,0.02,-0.02,31.987256,80.539811,0.062939,0.031334,0.0,0.0,1.0
11036,2020-05-14 23:45:00,27.75,27.81,27.75,27.79,25,0.06,0.06,0.00,0.04,30.380901,78.011621,0.062072,0.030428,0.0,0.0,2.0
11037,2020-05-14 23:50:00,27.78,27.78,27.77,27.77,3,0.01,0.00,0.01,-0.02,30.020558,75.365458,0.061933,0.029396,0.0,0.0,1.0


In [12]:
state_vars = ['open', 'high', 'low']
levels = [sorted(df[var].unique()) for var in state_vars]

In [13]:
num_bins = 3
for i, var_name in enumerate(state_vars):
    if len(levels[i]) > num_bins:
        df[var_name] = pd.qcut(df[var_name], q=num_bins, labels=False)
        levels[i] = sorted(df[var_name].unique())
    else:
        df[var_name] = df[var_name].astype('category').cat.codes
        levels[i] = sorted(df[var_name].unique())

In [14]:
all_regimes = list(np.array(np.meshgrid(*levels)).T.reshape(-1, len(state_vars)))
num_regimes = len(all_regimes)

def get_regime(row):
    return tuple(row[var] for var in state_vars)

df['regime'] = df.apply(get_regime, axis=1).astype(object)

In [15]:
def log_likelihood(data, shape, scale):
    if scale <= 0 or shape <= 0:
        return -np.inf
    return np.sum(gamma.logpdf(data, a=shape, scale=scale))


def log_prior(shape, scale):
    if scale <= 0 or shape <= 0:
        return -np.inf
    return stats.expon.logpdf(shape, scale=10) + stats.expon.logpdf(scale, scale=10)


def log_posterior(data, shape, scale):
    return log_likelihood(data, shape, scale) + log_prior(shape, scale)

In [16]:
def metropolis_hastings(data, initial_params, log_posterior_func, iterations, proposal_width):
    samples = np.zeros((iterations + 1, len(initial_params)))
    samples[0] = initial_params
    accepted = 0

    current_log_posterior = log_posterior_func(data, *initial_params)

    for i in range(iterations):
        proposed_params = samples[i] + np.random.normal(0, proposal_width, size=len(initial_params))

        proposed_log_posterior = log_posterior_func(data, *proposed_params)
        
        log_diff = proposed_log_posterior - current_log_posterior
        acceptance_prob = 1 if log_diff >= 0 else np.exp(log_diff) 

        if np.random.rand() < acceptance_prob:
            samples[i + 1] = proposed_params
            current_log_posterior = proposed_log_posterior
            accepted += 1
        else:
            samples[i + 1] = samples[i]

    return samples, accepted / iterations


In [19]:
all_regime_samples = {}
iterations = 100
proposal_width = 0.5

for regime, samples in all_regime_samples.items():
    print(f"Key type: {type(regime)}, Value type: {type(samples)}")

for i, regime in enumerate(all_regimes):
    regime = tuple(regime)  
    regime_data = df[df['regime'].apply(lambda x: all(x[j] == regime[j] for j in range(len(regime)) if len(x) == len(regime)))]['volume'].values
    if len(regime_data) > 0:
        initial_params = (1.0, 1.0)
        samples, acceptance_rate = metropolis_hastings(regime_data, initial_params, log_posterior, iterations, proposal_width)
        all_regime_samples[regime] = samples 
        print(f"Regime {regime}: Acceptance Rate = {acceptance_rate:.3f}")

Regime (0, 0, 0): Acceptance Rate = 0.510
Regime (0, 1, 0): Acceptance Rate = 0.400
Regime (1, 0, 0): Acceptance Rate = 0.560
Regime (1, 1, 0): Acceptance Rate = 0.470
Regime (0, 0, 1): Acceptance Rate = 0.420
Regime (0, 1, 1): Acceptance Rate = 0.580
Regime (1, 0, 1): Acceptance Rate = 0.550
Regime (1, 1, 1): Acceptance Rate = 0.490
Regime (1, 2, 1): Acceptance Rate = 0.530
Regime (2, 1, 1): Acceptance Rate = 0.410
Regime (2, 2, 1): Acceptance Rate = 0.540
Regime (1, 1, 2): Acceptance Rate = 0.590
Regime (1, 2, 2): Acceptance Rate = 0.560
Regime (2, 1, 2): Acceptance Rate = 0.540
Regime (2, 2, 2): Acceptance Rate = 0.540
Unique regimes in data: 15
